In [1]:
import pandas as pd

In [2]:
#loading the data
train=pd.read_csv("data/train.csv")
test=pd.read_csv("data/test.csv")

In [3]:
#checking out the data-set
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
age               32561 non-null int64
workclass         30725 non-null object
fnlwgt            32561 non-null int64
education         32561 non-null object
education.num     32561 non-null int64
marital.status    32561 non-null object
occupation        30718 non-null object
relationship      32561 non-null object
race              32561 non-null object
sex               32561 non-null object
capital.gain      32561 non-null int64
capital.loss      32561 non-null int64
hours.per.week    32561 non-null int64
native.country    31978 non-null object
target            32561 non-null object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [5]:
print("Number of rows and columns in training data: ",train.shape)
print("Number of rows and columns in test data: ",test.shape)

Number of rows and columns in training data:  (32561, 15)
Number of rows and columns in test data:  (16281, 15)


In [6]:
#ok let me see what does the data look like
train.head()

age          workclass  fnlwgt   education  education.num  \
0   39          State-gov   77516   Bachelors             13   
1   50   Self-emp-not-inc   83311   Bachelors             13   
2   38            Private  215646     HS-grad              9   
3   53            Private  234721        11th              7   
4   28            Private  338409   Bachelors             13   

        marital.status          occupation    relationship    race      sex  \
0        Never-married        Adm-clerical   Not-in-family   White     Male   
1   Married-civ-spouse     Exec-managerial         Husband   White     Male   
2             Divorced   Handlers-cleaners   Not-in-family   White     Male   
3   Married-civ-spouse   Handlers-cleaners         Husband   Black     Male   
4   Married-civ-spouse      Prof-specialty            Wife   Black   Female   

   capital.gain  capital.loss  hours.per.week  native.country  target  
0          2174             0              40   United-States   <=50K  
1             0             0              13   United-States   <=50K  
2             0             0              40   United-States   <=50K  
3             0             0              40   United-States   <=50K  
4             0             0              40            Cuba   <=50K

#### Let's check for missing values in the data(if present)

In [10]:
train_missing=train.shape[0]-train.dropna().shape[0]
print("%d rows have missing values in the training data" % train_missing)
test_missing=test.shape[0]-test.dropna().shape[0]
print("%d rows have missing values in the test data" % test_missing)

2399 rows have missing values in the training data
1221 rows have missing values in the test data


#### Digging deeper and finding out which columns have missing values.

In [11]:
train.isnull().sum()

age                  0
workclass         1836
fnlwgt               0
education            0
education.num        0
marital.status       0
occupation        1843
relationship         0
race                 0
sex                  0
capital.gain         0
capital.loss         0
hours.per.week       0
native.country     583
target               0
dtype: int64

Counting the number of unique values from cahracter variables

In [12]:
cat = train.select_dtypes(include=['O'])
cat.apply(pd.Series.nunique)

workclass          8
education         16
marital.status     7
occupation        14
relationship       6
race               5
sex                2
native.country    41
target             2
dtype: int64

Since missing values are found in all the 3 character variables, let's impute these missing values with their respective modes.

In [15]:
#Workclass
train.workclass.value_counts(sort=True)
train.workclass.fillna('Private',inplace=True)

#occupation
train.occupation.value_counts(sort=True)
train.occupation.fillna('Prof-speciality',inplace=True)

#Native Country 
train['native.country'].value_counts(sort=True)
train['native.country'].fillna('United-States',inplace=True)


Let's check if there are any missing values lift

In [16]:
train.isnull().sum()

age               0
workclass         0
fnlwgt            0
education         0
education.num     0
marital.status    0
occupation        0
relationship      0
race              0
sex               0
capital.gain      0
capital.loss      0
hours.per.week    0
native.country    0
target            0
dtype: int64

Now checking the target variable if this data is imbalanced or not

In [17]:
train.target.value_counts()/train.shape[0]  #proportion of target variable

 <=50K    0.75919
 >50K     0.24081
Name: target, dtype: float64

ok. So we can clearly see that 75% of our data belongs to less than $50k class. This implies that even if we took our wildest guess at target prediction as less than 50k, we will get accuracy of 75%. 
Will now create a cross tab of the target variable with education. With this, we will try to understand the influence of education on the target variable.

In [25]:
#every figure below is in %age
crosstab=pd.crosstab(train.education,train.target,margins=True)/train.shape[0]
crosstab*100

target             <=50K       >50K         All
education                                      
 10th           2.674979   0.190412    2.865391
 11th           3.424342   0.184270    3.608612
 12th           1.228463   0.101348    1.329812
 1st-4th        0.497528   0.018427    0.515955
 5th-6th        0.973557   0.049139    1.022696
 7th-8th        1.861122   0.122846    1.983969
 9th            1.495654   0.082921    1.578576
 Assoc-acdm     2.463069   0.813857    3.276926
 Assoc-voc      3.135653   1.108688    4.244341
 Bachelors      9.625012   6.821044   16.446055
 Doctorate      0.328614   0.939775    1.268389
 HS-grad       27.106047   5.144191   32.250238
 Masters        2.346365   2.945241    5.291607
 Preschool      0.156629   0.000000    0.156629
 Prof-school    0.469887   1.299100    1.768987
 Some-college  18.132121   4.259697   22.391818
All            75.919044  24.080956  100.000000

So, clearly that out of 75% of people <=50k salary